# Загрузка библиотек

In [1]:
import pandas as pd
import numpy as np

# Определение констант

In [2]:
data_path = 'data/'

# Загрузка данных

In [3]:
sales = pd.read_parquet(data_path + 'sales.parquet')
shops = pd.read_parquet(data_path + 'shops.parquet')

# Трансформация данных

In [4]:
sales.head()

,date,shop_id,owner,number_of_counters,goods_type,total_items_sold
0,2146-01-01,0,Рейдеры,4,Съедобный хлам,6.0
1,2146-01-01,0,Рейдеры,4,Хлам,26.0
2,2146-01-01,0,Рейдеры,4,Бензак,10537.0
3,2146-01-01,1,Рейдеры,5,Съедобный хлам,17.0
4,2146-01-01,1,Рейдеры,5,Хлам,9.0


Данные в длинном формате (1 наблюдение - продажи 1 товара в 1 магазине за 1 день)

In [5]:
sales.groupby(['date', 'shop_id']).apply(lambda x: len(x['number_of_counters'].unique())).sort_values()

date        shop_id
2146-01-01  0          1
2147-05-02  264        1
            265        1
            266        1
            267        1
                      ..
2146-09-01  194        1
            195        1
            196        1
            198        1
2147-12-31  844        1
Length: 615736, dtype: int64

В кадом магазине в 1 день 1 количество прилавков для всех товаров, поэтому может перейти к широкому формату

In [6]:
# Приводим данные от длинного формата к широкому
sales_pivoted = pd.pivot_table(sales[['date', 'shop_id', 'goods_type', 'total_items_sold']],
                               index = ['date', 'shop_id'],
                               columns = 'goods_type')

In [7]:
# Исправляем название колонок и индексы
sales_pivoted.columns = sales_pivoted.columns.get_level_values(1)
sales_pivoted = sales_pivoted.reset_index()
sales_pivoted.columns.name = ''

In [8]:
sales_pivoted.head()

,date,shop_id,Бензак,Броня и одежда,Жидкости для тачки,Медпрепараты и еда,Модификации тачки,Оружие,Патроны,Солярка,Съедобный хлам,Хлам,Ядер-Кола
0,2146-01-01,0,10537.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,26.0,NaN
1,2146-01-01,1,5510.0,2.0,NaN,NaN,NaN,NaN,NaN,238.0,17.0,9.0,NaN
2,2146-01-01,2,5893.0,16.0,NaN,NaN,150.0,17.0,671.0,629.0,115.0,121.0,37.0
3,2146-01-01,3,14627.0,2.0,NaN,NaN,NaN,NaN,NaN,822.0,20.0,63.0,NaN
4,2146-01-01,4,15525.0,13.0,21.0,NaN,15.0,NaN,449.0,1011.0,281.0,68.0,146.0


In [9]:
# Добавляем информацию о количестве прилавков
sales = sales[['date', 'shop_id', 'number_of_counters']].drop_duplicates(['date', 'shop_id']).reset_index(drop = True)
sales_pivoted = pd.merge(sales_pivoted, sales, on = ['date', 'shop_id'])

In [10]:
sales_pivoted.head()

,date,shop_id,Бензак,Броня и одежда,Жидкости для тачки,Медпрепараты и еда,Модификации тачки,Оружие,Патроны,Солярка,Съедобный хлам,Хлам,Ядер-Кола,number_of_counters
0,2146-01-01,0,10537.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,26.0,NaN,4
1,2146-01-01,1,5510.0,2.0,NaN,NaN,NaN,NaN,NaN,238.0,17.0,9.0,NaN,5
2,2146-01-01,2,5893.0,16.0,NaN,NaN,150.0,17.0,671.0,629.0,115.0,121.0,37.0,4
3,2146-01-01,3,14627.0,2.0,NaN,NaN,NaN,NaN,NaN,822.0,20.0,63.0,NaN,4
4,2146-01-01,4,15525.0,13.0,21.0,NaN,15.0,NaN,449.0,1011.0,281.0,68.0,146.0,10


Проверим промежутки продаж в магазинах

In [11]:
sales_pivoted.groupby('shop_id').apply(lambda x: x['date'].min()).sort_values()

shop_id
0     2146-01-01
556   2146-01-01
557   2146-01-01
558   2146-01-01
559   2146-01-01
         ...    
285   2146-01-01
286   2146-01-01
287   2146-01-01
263   2146-01-01
844   2146-01-01
Length: 845, dtype: datetime64[ns]

In [12]:
sales_pivoted.groupby('shop_id').apply(lambda x: x['date'].max()).sort_values()

shop_id
178   2147-11-30
181   2147-12-02
182   2147-12-19
180   2147-12-24
179   2147-12-24
         ...    
289   2147-12-31
290   2147-12-31
291   2147-12-31
267   2147-12-31
844   2147-12-31
Length: 845, dtype: datetime64[ns]

Данные о продажах в разных магазинах доступны за разные промежутки. Ограничим все одним промежутком

In [13]:
# Ограничиваем данные о продажах этим промежутком
# В этот промежуток есть информация о продажах всех товаров
sales_pivoted = sales_pivoted[(sales_pivoted['date']>=pd.to_datetime('2146-01-01 00:00:00'))\
                              & (sales_pivoted['date']<=pd.to_datetime('2147-11-30 00:00:00'))]

Проверим, что в даннных о продажах нет пропущенных дней

In [14]:
# Отсортируем по дате
sales_pivoted = sales_pivoted.sort_values(['shop_id', 'date']).reset_index(drop = True)

In [15]:
# Предыдущий день исходя из календаря.
sales_pivoted['date_previous_real'] = sales_pivoted.groupby('shop_id')['date'].apply(lambda x: x - pd.Timedelta('1 day'))
# Предыдущий день, исходя из наблюдений
sales_pivoted['date_previous_fact'] = sales_pivoted.groupby('shop_id')['date'].apply(lambda x: x.shift(1))

In [16]:
sales_pivoted_dates = sales_pivoted.dropna(subset = ['date_previous_fact'])[['date', 
                                                                             'shop_id', 
                                                                             'date_previous_fact', 
                                                                             'date_previous_real']]

In [17]:
sales_pivoted_dates[sales_pivoted_dates['date_previous_fact'] != sales_pivoted_dates['date_previous_real']]

,date,shop_id,date_previous_fact,date_previous_real
1344,2147-10-10,1,2147-10-07,2147-10-09
1908,2147-05-30,2,2147-05-27,2147-05-29
2339,2146-09-05,3,2146-09-03,2146-09-04
3310,2147-06-06,4,2147-06-03,2147-06-05
5799,2146-08-05,8,2146-08-02,2146-08-04
...,...,...,...,...
578653,2146-06-08,829,2146-06-06,2146-06-07
578847,2146-12-21,829,2146-12-18,2146-12-20
579140,2147-10-17,829,2147-10-09,2147-10-16
579155,2147-11-06,829,2147-10-31,2147-11-05


Для некоторых магазинов есть пропущенные даты. Заполним их для упрощения анализа

In [18]:
sales_pivoted = sales_pivoted.drop(['date_previous_fact', 'date_previous_real'], axis = 'columns')

In [19]:
def filling_gaps(x):

    ''' Возвращает датасет с заполненными датами '''

    dates = list(pd.date_range('2146-01-01 00:00:00', '2147-11-30 00:00:00'))
    shop_id = x['shop_id'].unique()[0]
    dates = pd.DataFrame({'date': dates, 'shop_id': shop_id})
    x = pd.merge(x, dates, on = ['date', 'shop_id'], how = 'right')

    return x

In [20]:
sales_pivoted = sales_pivoted.groupby('shop_id').apply(filling_gaps).reset_index(drop = True)
sales_pivoted = sales_pivoted.sort_values(['shop_id', 'date'])

In [21]:
sales_pivoted.head()

,date,shop_id,Бензак,Броня и одежда,Жидкости для тачки,Медпрепараты и еда,Модификации тачки,Оружие,Патроны,Солярка,Съедобный хлам,Хлам,Ядер-Кола,number_of_counters
0,2146-01-01,0,10537.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,26.0,NaN,4.0
1,2146-01-02,0,14107.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,5.0,NaN,4.0
2,2146-01-03,0,16092.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,18.0,17.0,NaN,4.0
3,2146-01-04,0,18050.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,30.0,70.0,NaN,4.0
4,2146-01-05,0,18456.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,6.0,NaN,4.0
